<a href="https://colab.research.google.com/github/Chocoding1/Machine_Learning_Deep_Learning/blob/main/05_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **트리의 앙상블**

## 정형 데이터와 비정형 데이터

- 정형 데이터 : csv나 데이터베이스, 엑셀 등과 같이 어떤 구조로 되어 있는 데이터
- 비정형 데이터 : 텍스트, 사진, 음악 등과 같이 어떤 구조가 없는 데이터 / 데이터베이스나 엑셀로 표현하기 어려움<br>

**앙상블 학습** : 정형 데이터를 다루는 데 가장 뛰어난 성과를 내는 알고리즘 / 대부분 결정 트리를 기반으로 만들어져 있다.<br>
note) 비정형 데이터를 다루는 데 사용되는 알고리즘은 신경망 알고리즘이다.

## 랜덤 포레스트

앙상블 학습의 대표 주자 중 하나로 안정적인 성능 덕에 널리 사용됨<br>
이름처럼 랜덤 포레스트는 결정 트리를 랜덤하게 만들어 결정 트리의 **숲**을 만든다. 그리고 각 결정 트리의 예측을 사용해 최종 예측을 만든다.<br>
랜덤 포레스트는 각 트리를 훈련하기 위한 데이터를 랜덤하게 만드는데, 우리가 입력한 훈련 데이터에서 랜덤하게 샘플을 추출하여 훈련 데이터를 만든다.(샘플은 중복되어 추출될 수 있음)<br>
이렇게 만들어진 샘플을 **부트스트램 샘플**이라고 한다. 기본적으로 부트스트랩 샘플은 훈련 세트의 크기와 같게 만든다.<br>

랜덤 포레스트는 각 노드를 분할할 때 전체 특성 중에서 일부 특성을 무작위로 고른 다음 이 중에서 최선의 분할을 찾는다. <br>
분류 모델인 RandomRorestClassifier는 기본적으로 전체 특성의 개수의 제곱근만큼의 특성을 선택한다. 즉 4개의 특성이 있다면 노드마다 2개의 특성을 랜덤하게 선택하여 사용하는 것(회귀 모델인 RandomForestRegressor는 전체 특성 사용)<br>

랜덤 포레스트는 랜덤하게 선택한 샘플과 특성을 사용하기 때문에 훈련 세트에 과대적합되는 것을 막아주고, 일반화 성능을 높여주며 검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있다.(종종 기본 매개변수 설정만으로도 아주 좋은 결과 나올 때도 있음)

note) 부트스트랩이란?<br>
데이터 세트에서 중복을 허용하여 데이터를 샘플링하는 방식을 의미<br>



RandomForestClassifier 클래스를 활용하여 앞서 했던 화이트 와인 분류를 다시 해보자

In [2]:
# 와인 데이터셋 불러와 훈련 세트와 테스트 세트로 분할
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

cross_validate() 함수로 교차 검증을 수행할 텐데, RandomForestClassifier는 기본적으로 100개의 결정 트리를 사용하므로 n_jobs 매개변수를 -1로 지정하여 모든 CPU 코어를 사용하는 것이 좋다.

In [6]:
# 교차 검증 수행
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1) # return_train_score : True로 지정하면 검증 점수뿐만 아니라 훈련 세트에 대한 점수도 같이 반환(기본값 False)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


-> 과대 적합을 보인다.

랜덤 포레스트는 결정 트리의 앙상블이기 때문에 DecisionTreeClassifier가 제공하는 중요한 매개변수를 모두 제공한다. (criterion, max_depth, max_features, min_samples_split, min_impurity_decrease, min_samples_leaf 등)<br>
또한 결정 트리의 큰 장점 중 하나인 특성 중요도를 계산한다. 랜덤 포레스트의 특성 중요도는 각 결정 트리의 특성 중요도를 취합한 것이다.

In [7]:
# 랜덤 포레스트 모델 훈련 후 특성 중요도 출력
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


RandomForestClassifier에는 자체적으로 모델을 평가하는 점수를 얻을 수 있는 기능이 있다.<br>
랜덤 포레스트는 훈련 세트에서 중복을 허용하여 부트스트랩 샘플을 만들어 결정 트리를 훈련하는데, 이 때 부트스트랩 샘플에 포함되지 않고 남는 샘플이 있다. 이런 샘플을 **OOB(out of bag)** 샘플이라고 한다.<br>
이 남는 샘플을 사용하여 부트스트랩 샘플로 훈련한 결정 트리를 평가하는 것이다. 검증 세트의 역할을 하는 것

해당 점수를 얻으려면 RandomForestClassifier 클래스의 oob_score 매개변수를 True로 지정해야 한다.(기본값 False)<br>
이러면 랜덤 포레스트는 각 결정 트리의 OOB 점수를 평균하여 출력한다.

In [8]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


OOB 점수를 활용하면 교차 검증을 대신할 수 있어서 결과적으로 훈련 세트에 더 많은 샘플을 사용할 수 있다.

## 엑스트라 트리

랜덤 포레스트와 비슷<br>
- 기본적으로 100개의 결정 트리 훈련
- 랜덤 포레스트와 동일하게 결정 트리가 제공하는 대부분의 매개변수 지원
- 전체 특성 중에 일부 특성을 랜덤하게 선택하여 노드를 분할하는 데 사용

랜덤 포레스트와 차이점<br>
- 엑스트라 트리는 부트스트랩 샘플을 사용하지 않는다. 즉 각 결정 트리를 만들 때 전체 훈련 세트를 사용
- 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할한다.
- 이전에 DecisionTreeClassifier의 splitter 매개변수를 'random'으로 지정해봤는데, 엑스트라 트리가 사용하는 결정 트리가 바로 splitter='random'인 결정 트리이다.

장점 : 노드를 랜덤하게 분할하기 때문에 계산 속도가 빠르다.<br>
단점 : 엑스트라 트리가 무작위성이 좀 더 크기 때문에 랜덤 포레스트보다 더 많은 결정 트리를 훈련해야 한다.

하나의 결정 트리에서 특성을 무작위로 분할한다면 성능이 낮아지겠지만 많은 트리를 앙상블하기 때문에 과대적합을 막고 검증 세트의 점수를 높이는 효과가 있다.

In [9]:
# 엑스트라 트리 모델의 교차 검증 점수 확인
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


-> 엑스트라 트리도 과대 적합을 보인다.<br>

In [13]:
# 특성 중요도 확인(엑스트라 트리 또한 특성 중요도 제공)
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


엑스트라 트리의 회귀 버전은 ExtraTreesRegressor

## 그레이디언트 부스팅

깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블하는 방법<br>
사이킷런의 GradientBoostingClassifier는 기본적으로 깊이가 3인 결정 트리 100개 사용<br>
깊이가 얕은 결정 트리를 사용하기 때문에 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대할 수 있다.<br>

이름이 그레이디언트인 만큼 경사 하강법을 사용하여 트리를 앙상블에 추가한다. 분류에서는 로지스틱 손실 함수를 사용하고 회귀에서는 평균 제곱 오차 함수를 사용한다.<br>

그레이디언트 부스팅은 결정 트리를 계속 추가하면서 가장 낮은 곳을 찾아 이동한다. 경사 하강법이 가장 낮은 곳으로 조금씩 천천히 이동하는 것처럼 그레이디언트 부스팅도 조금씩 움직여야 한다. 그래서 깊이가 얕은 트리를 사용하는 것<br>
학습률 매개변수를 통해 속도를 조절한다.

In [12]:
# 그레이디언트 부스팅을 사용해 와인 데이터셋의 교차 검증 점수 확인
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


-> 그레이디언트 부스팅은 과대적합을 보이지 않는다.<br>

그레이디언트 부스팅은 결정 트리의 개수를 늘려도 과대적합에 매우 강하다. 학습률을 증가시키고 트리의 개수를 늘리면 조금 더 성능이 향상될 수 있다.

In [15]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42) # n_estimators : 결정 트리 개수 지정 / learning_rate 기본값 : 0.1
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [17]:
# 특성 중요도 확인(그레이디언트 부스팅도 특성 중요도 제공)
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


추가) subsample : 트리 훈련에 사용할 훈련 세트의 비율을 정하는 매개변수(기본값 : 1.0 - 전체 훈련 세트 사용) / 1보다 작으면 훈련 세트의 일부를 사용(확률적/미니배치 경사 하강법과 비슷)

일반적으로 그레이디언트 부스팅이 랜덤 포레스트보다 조금 더 높은 성능을 얻을 수 있지만, 순서대로 트리를 추가하기 때문에 훈련 속도가 느리다. 때문에 GradientBoostingClassifier에는 n_jobs 매개변수가 없다.

## 히스토그램 기반 그레이디언트 부스팅

그레이디언트 부스팅의 속도와 성능을 더욱 개선한 것으로, 정형 데이터를 다루는 머신러닝 알고리즘 중에 가장 인기가 높다.<br>

히스토그램 기반 그레이디언트 부스팅은 먼저 입력 특성을 256개의 구간으로 나눈다. 따라서 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있다.<br>

히스토그램 기반 그레이디언트 부스팅은 256개의 구간 중에서 하나를 떼어 놓고 누락된 값을 위해 사용한다. 따라서 입력에 누락된 특성이 있어도 이를 따로 전처리할 필요 x<br>

사이킷런의 히스토그램 기반 그레이디언트 부스팅 클래스는 HistGradientBoostingClassifier<br>
일반적으로 HistGradientBoostingClassifier는 기본 매개변수에서 안정적인 성능을 얻을 수 있다.<br>
HistGradientBoostingClassifier에는 트리의 개수를 지정하는 데 n_estimators 대신 부스팅 반복 횟수를 지정하는 max_iter를 사용한다. 성능을 높이려면 max_iter 매개변수를 테스트해보면 될 듯 하다.

사이킷런의 히스토그램 기반 그레이디언트 부스팅은 아직 테스트 과정에 있다고 한다. 따라서 해당 클래스를 사용하려면 sklearn.experimental 패키지 아래에 있는 enaable_hist_gradient_boosting 모듈을 임포트해야 한다. -> 경고 뜬 거 보니까 이젠 안정적이라고 굳이 임포트 안 해도 되는 것 같다.

In [23]:
# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


-> 과대적합을 잘 억제하면서 그레이디언트 부스팅보다 조금 더 높은 성능 제공

히스토그램 기반 그레이디언트 부스팅의 특성 중요도를 계산할 때 permutation_importance() 함수를 사용한다.<br>
해당 함수는 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰하여 어떤 특성이 중요한지를 계산한다.<br>
훈련 세트뿐만 아니라 테스트 세트에도 적용할 수 있고 사이킷런에서 제공하는 추정기 모델에 모두 사용 가능하다.

In [30]:
# 히스토그램 기반 그레이디언트 부스팅 모델 훈련 후 훈련 세트에서 특성 중요도 계산
# n_repeats : 랜덤하게 섞을 횟수 지정(기본값 : 5)
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


permutation_importance() 함수가 반환하는 객체는 특성 중요도(importances), 평균(importancces_mean), 표준 편차(importances_std)를 담고 있다.

In [31]:
# 위에서 훈련한 모델로 테스트 세트에서 특성 중요도 계산
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


테스트 세트의 결과, 그레이디언트 부스팅과 비슷하게 조금 더 당도에 집중하고 있는 것을 알 수 있다.<br>

이런 분석을 토해 모델을 실전에 투입했을 때 어떤 특성에 관심을 둘지 예상할 수 있다.

In [33]:
# HsitGradientBoostingClassifier을 사용해 테스트 세트에서의 성능을 최종적으로 확인
hgb.score(test_input, test_target)

0.8723076923076923

사이킷런 외에도 히스토그램 기반 그레이디언트 부스팅 알고리즘을 구현한 라이브러리가 여러 개 있다.<br>

가장 대표적인 라이브러리가 **XGBoost**<br>

사이킷런의 cross_validate() 함수와 함께 사용할 수 있다. 또한 다양한 부스팅 알고리즘을 지원하는데, tree_method 매개변수를 'hist'로 지정하면 히스토그램 기반 그레이디언트 부스팅을 사용할 수 있다.

In [34]:
# XGBoost를 사용해 와인 데이터셋의 교차 검증 점수 확인
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


또 다른 히스토그램 기반 그레이디언트 부스팅 라이브러리는 **LightGBM**(Microsoft)

In [35]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

In [36]:
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
